In [1]:
from __future__ import print_function

import sys
import os
import time
import numpy as np
import mxnet as mx
from mxnet import nd,autograd,init
from mxnet.gluon import Trainer,data as gdata,loss as gloss,nn
from d2l import mxnet as d2l

from dataset import ShapeNet3D
from model import BlockOuterNet, RenderNet
from criterion import BatchIoU
from misc import clip_gradient, decode_multiple_block
from options import options_guided_adaptation,options_train_generator,options_train_executor
from visualization.util_vtk import visualization

In [2]:
def train(epoch, train_loader, generator, executor, criterion, optimizer, opt,ctx):
    """
    one epoch guided adaptation
    """

    for idx, data in enumerate(train_loader):
        start = time.time()
        shapes = data.as_in_context(ctx)
        raw_shapes = data
        shapes = shapes.expand_dims(axis = 1)
        with autograd.record():
            pgms, params = generator.decode(shapes)
            
            # truly rendered shapes
            pgms_int = nd.round(pgms).astype('int64')
            params_int = nd.round(params).astype('int64')
           

            # neurally rendered shapes
            pgms = nd.exp(pgms)
            bsz, n_block, n_step, n_vocab = pgms.shape
            pgm_vector = pgms.reshape(bsz*n_block, n_step, n_vocab)
            bsz, n_block, n_step, n_param = params.shape
            param_vector = params.reshape(bsz*n_block, n_step, n_param)
            index = (n_step - 1) * nd.ones(bsz * n_block).astype('int64')
            index = index.as_in_context(ctx)
            pred = executor(pgm_vector, param_vector, index)
            pred = nd.softmax(pred,axis = 1)
            #print(pred.shape)
            pred = pred[:, 1]
            pred = pred.reshape(bsz, n_block, 32, 32, 32)
            
            rec = nd.max(pred, axis=1)
            rec1 = nd.log(rec+ 1e-11)
            rec0 = nd.log(1 - rec+1e-11)
            gt = shapes.squeeze().astype('int64')
            loss = -nd.where(gt,rec1,rec0).mean(axis = (1,2,3))

        loss.backward()
        optimizer.step(loss.shape[0],ignore_stale_grad=True)
        l = loss.mean().asscalar()
        
        rendered_shapes = decode_multiple_block(pgms_int, params_int)
        rendered_shapes = nd.from_numpy(rendered_shapes).astype('float32').as_in_context(mx.cpu())
        IoU2= BatchIoU(raw_shapes,rendered_shapes)
        reconstruction = (rec.as_in_context(mx.cpu())>0.5).astype('float32')
        IoU1 = BatchIoU(reconstruction, raw_shapes)
        #print("IoU1:",IoU1,IoU2)
        IoU1 = IoU1.mean()
        IoU2 = IoU2.mean()
        

        end = time.time()

        if idx % opt.info_interval == 0:
            print("Train: epoch {} batch {}/{}, loss = {:.3f}, IoU1 = {:.3f}, IoU2 = {:.3f}, time = {:.3f}"
                  .format(epoch, idx, len(train_loader), l, IoU1, IoU2, end - start))
            sys.stdout.flush()


def validate(epoch, val_loader, generator, opt,ctx,gen_shape=False):
    """
    evaluate program generator, in terms of IoU
    """
    generated_shapes = []
    original_shapes = []
    for idx, data in enumerate(val_loader):
        start = time.time()
        shapes = data.as_in_context(ctx)
        shapes = nd.expand_dims(shapes, axis=1)
        with autograd.train_mode():
            out = generator.decode(shapes)

        end = time.time()
        
        if gen_shape:
            out_1 = nd.round(out[0]).astype('int64')
            out_2 = nd.round(out[1]).astype('int64')
            generated_shapes.append(decode_multiple_block(out_1, out_2).astype("float32"))
            original_shapes.append(data.asnumpy())

        if idx % opt.info_interval == 0:
            print("Test: epoch {} batch {}/{}, time={:.3f}"
                  .format(epoch, idx, len(val_loader), end - start))

    if gen_shape:
        generated_shapes = np.concatenate(generated_shapes, axis=0)
        original_shapes = np.concatenate(original_shapes, axis=0)

    return generated_shapes, original_shapes




In [3]:
# get options
opt = options_guided_adaptation.parse()
opt_gen = options_train_generator.parse()
opt_exe = options_train_executor.parse()
print('===== arguments: guided adaptation =====')
for key, val in vars(opt).items():
    print("{:20} {}".format(key, val))
print('===== arguments: guided adaptation =====')

if not os.path.isdir(opt.save_folder):
    os.makedirs(opt.save_folder)

# build loaders
train_set = ShapeNet3D(opt.train_file)
train_loader = gdata.DataLoader(
    dataset=train_set,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.num_workers
)

val_set = ShapeNet3D(opt.val_file)
val_loader = gdata.DataLoader(
    dataset=val_set,
    batch_size=opt.batch_size,
    shuffle=False,
    num_workers=opt.num_workers
)

===== arguments: guided adaptation =====
learning_rate        2e-05
weight_decay         1e-05
beta1                0.9
beta2                0.999
epochs               60
grad_clip            0.1
info_interval        10
save_interval        5
batch_size           32
num_workers          4
data_folder          ./data/
cls                  sofa
p_gen_path           ./model/ckpts_program_generator/program_generator.t7
p_exe_path           ./model/ckpts_program_executor/program_executor.t7
model_name           GA
save_folder          ./model/ckpts_GA_sofa
train_file           ./data/sofa_training.h5
val_file             ./data/sofa_testing.h5
===== arguments: guided adaptation =====


In [4]:
def visual(path,epoch,gen_shapes,file_name,nums_samples):
    data = gen_shapes.transpose((0, 3, 2, 1))
    data = np.flip(data, axis=2)
    num_shapes = data.shape[0]
    for i in range(min(nums_samples,num_shapes)):
        voxels = data[i]
        save_name = os.path.join(path, file_name.format(epoch,i))
        visualization(voxels,
                      threshold=0.1,
                      save_name=save_name,
                      uniform_size=0.9)

In [ ]:
ctx = d2l.try_gpu()

# load program generator
generator = BlockOuterNet(opt_gen)
generator.init_blocks(ctx)
generator.load_parameters("model/model of blockouternet")


# load program executor
executor = RenderNet(opt_exe)
executor.initialize(init = init.Xavier(),ctx = ctx)
executor.load_parameters("model/model of executor")

def set_bn_eval(m):
    if m.prefix[:9]=='batchnorm':
        m._kwargs['use_global_stats']=True      
executor.apply(set_bn_eval)


# build loss functions
criterion = gloss.SoftmaxCrossEntropyLoss(axis=1,from_logits=True)

optimizer = Trainer(generator.collect_params(),"adam",
                    {"learning_rate":opt.learning_rate,"wd":opt.weight_decay,
                     'beta1':opt.beta1, 'beta2':opt.beta2,'clip_gradient': opt.grad_clip})

'''
optimizer = Trainer(generator.collect_params(),"sgd",
                {"learning_rate":opt.learning_rate*30,"momentum":0.1,
                 "wd":opt.weight_decay,'clip_gradient': opt.grad_clip})
'''
print("###################")
print("testing")
gen_shapes, ori_shapes = validate(0, val_loader, generator, opt,ctx,gen_shape=True)
if os.path.exists('imgs of chairs/adaption/{}/'.format(opt.cls)) == False:
    os.mkdir('imgs of chairs/adaption/{}/'.format(opt.cls));
visual('imgs of chairs/adaption/{}/'.format(opt.cls),0,ori_shapes,'GT {}-{}.png',8)
visual('imgs of chairs/adaption/{}/'.format(opt.cls),0,gen_shapes,'epoch{}-{}.png',8)

gen_shapes = nd.from_numpy(gen_shapes)
ori_shapes = nd.from_numpy(ori_shapes)
#print(gen_shapes.dtype,ori_shapes.dtype)
#print("done",ori_shapes.shape,gen_shapes.shape)


IoU = BatchIoU(gen_shapes,ori_shapes)
#print(IoU)
print("iou: ", IoU.mean())


best_iou ,best_epoch= 0,0
print(opt.epochs)
for epoch in range(1, opt.epochs+1):
    print("###################")
    print("adaptation")
    train(epoch, train_loader, generator, executor, criterion, optimizer, opt,ctx)
    print("###################")
    print("testing")
    gen_shapes, ori_shapes = validate(epoch, val_loader, generator, opt,ctx,gen_shape=True)
    visual('imgs of chairs/adaption/{}/'.format(opt.cls),epoch,gen_shapes,'epoch{}-{}.png',8)
    gen_shapes = nd.from_numpy(gen_shapes)
    ori_shapes = nd.from_numpy(ori_shapes)
    IoU = BatchIoU(gen_shapes,ori_shapes)
    print("iou: ", IoU.mean())
    

    if IoU.mean() >= best_iou:
        print('Saving best model')
        generator.save_parameters("model/generator of GA on shapenet "+opt.cls)
        best_iou = IoU.mean()
        best_epoch = epoch



###################
testing
Test: epoch 0 batch 0/20, time=0.750
Test: epoch 0 batch 10/20, time=0.184
1549 blocks filled
3114 blocks filled
1032 blocks filled
1870 blocks filled
804 blocks filled
1562 blocks filled
1167 blocks filled
1547 blocks filled
1454 blocks filled
5154 blocks filled
1970 blocks filled
1631 blocks filled
834 blocks filled
3342 blocks filled
1076 blocks filled
1504 blocks filled
iou:  0.37111053
60
###################
adaptation
Train: epoch 1 batch 0/80, loss = 0.509, IoU1 = 0.358, IoU2 = 0.375, time = 1.455
Train: epoch 1 batch 10/80, loss = 0.548, IoU1 = 0.397, IoU2 = 0.390, time = 0.756
Train: epoch 1 batch 20/80, loss = 0.543, IoU1 = 0.399, IoU2 = 0.399, time = 0.761
Train: epoch 1 batch 30/80, loss = 0.509, IoU1 = 0.351, IoU2 = 0.373, time = 0.761
Train: epoch 1 batch 40/80, loss = 0.578, IoU1 = 0.393, IoU2 = 0.397, time = 0.760
Train: epoch 1 batch 50/80, loss = 0.504, IoU1 = 0.421, IoU2 = 0.409, time = 0.761
Train: epoch 1 batch 60/80, loss = 0.411, IoU1 

1393 blocks filled
1114 blocks filled
iou:  0.59479636
Saving best model
###################
adaptation
Train: epoch 9 batch 0/80, loss = 0.220, IoU1 = 0.636, IoU2 = 0.623, time = 0.870
Train: epoch 9 batch 10/80, loss = 0.376, IoU1 = 0.512, IoU2 = 0.484, time = 0.774
Train: epoch 9 batch 20/80, loss = 0.151, IoU1 = 0.647, IoU2 = 0.646, time = 0.772
Train: epoch 9 batch 30/80, loss = 0.152, IoU1 = 0.641, IoU2 = 0.627, time = 0.774
Train: epoch 9 batch 40/80, loss = 0.194, IoU1 = 0.677, IoU2 = 0.673, time = 0.784
Train: epoch 9 batch 50/80, loss = 0.192, IoU1 = 0.584, IoU2 = 0.615, time = 0.776
Train: epoch 9 batch 60/80, loss = 0.215, IoU1 = 0.618, IoU2 = 0.601, time = 0.777
Train: epoch 9 batch 70/80, loss = 0.248, IoU1 = 0.616, IoU2 = 0.607, time = 0.776
###################
testing
Test: epoch 9 batch 0/20, time=0.193
Test: epoch 9 batch 10/20, time=0.187
1480 blocks filled
3530 blocks filled
858 blocks filled
1356 blocks filled
626 blocks filled
1851 blocks filled
1341 blocks filled

Train: epoch 17 batch 30/80, loss = 0.234, IoU1 = 0.607, IoU2 = 0.643, time = 0.770
Train: epoch 17 batch 40/80, loss = 0.254, IoU1 = 0.621, IoU2 = 0.620, time = 0.774
Train: epoch 17 batch 50/80, loss = 0.197, IoU1 = 0.620, IoU2 = 0.620, time = 0.775
Train: epoch 17 batch 60/80, loss = 0.259, IoU1 = 0.628, IoU2 = 0.623, time = 0.778
Train: epoch 17 batch 70/80, loss = 0.261, IoU1 = 0.566, IoU2 = 0.585, time = 0.774
###################
testing
Test: epoch 17 batch 0/20, time=0.197
Test: epoch 17 batch 10/20, time=0.190
1308 blocks filled
3140 blocks filled
897 blocks filled
1300 blocks filled
658 blocks filled
1821 blocks filled
1248 blocks filled
1318 blocks filled
iou:  0.6175754
Saving best model
###################
adaptation
Train: epoch 18 batch 0/80, loss = 0.241, IoU1 = 0.594, IoU2 = 0.576, time = 0.858
Train: epoch 18 batch 10/80, loss = 0.184, IoU1 = 0.618, IoU2 = 0.622, time = 0.776
Train: epoch 18 batch 20/80, loss = 0.251, IoU1 = 0.581, IoU2 = 0.568, time = 0.772
Train: ep

Train: epoch 25 batch 70/80, loss = 0.243, IoU1 = 0.573, IoU2 = 0.579, time = 0.780
###################
testing
Test: epoch 25 batch 0/20, time=0.196
Test: epoch 25 batch 10/20, time=0.191
1447 blocks filled
3227 blocks filled
1020 blocks filled
1819 blocks filled
726 blocks filled
2024 blocks filled
1245 blocks filled
1108 blocks filled
iou:  0.6276689
###################
adaptation
Train: epoch 26 batch 0/80, loss = 0.222, IoU1 = 0.596, IoU2 = 0.591, time = 0.858
Train: epoch 26 batch 10/80, loss = 0.261, IoU1 = 0.604, IoU2 = 0.607, time = 0.779
Train: epoch 26 batch 20/80, loss = 0.183, IoU1 = 0.595, IoU2 = 0.582, time = 0.778
Train: epoch 26 batch 30/80, loss = 0.243, IoU1 = 0.597, IoU2 = 0.602, time = 0.780
Train: epoch 26 batch 40/80, loss = 0.159, IoU1 = 0.605, IoU2 = 0.622, time = 0.777
Train: epoch 26 batch 50/80, loss = 0.161, IoU1 = 0.649, IoU2 = 0.656, time = 0.778
Train: epoch 26 batch 60/80, loss = 0.225, IoU1 = 0.597, IoU2 = 0.574, time = 0.782
Train: epoch 26 batch 70/8

Train: epoch 34 batch 10/80, loss = 0.240, IoU1 = 0.593, IoU2 = 0.567, time = 0.837
Train: epoch 34 batch 20/80, loss = 0.188, IoU1 = 0.609, IoU2 = 0.580, time = 0.806
Train: epoch 34 batch 30/80, loss = 0.251, IoU1 = 0.648, IoU2 = 0.616, time = 0.822
Train: epoch 34 batch 40/80, loss = 0.208, IoU1 = 0.603, IoU2 = 0.576, time = 0.794
Train: epoch 34 batch 50/80, loss = 0.164, IoU1 = 0.652, IoU2 = 0.633, time = 0.819
Train: epoch 34 batch 60/80, loss = 0.160, IoU1 = 0.644, IoU2 = 0.614, time = 0.821
Train: epoch 34 batch 70/80, loss = 0.131, IoU1 = 0.666, IoU2 = 0.641, time = 0.842
###################
testing
Test: epoch 34 batch 0/20, time=0.192
Test: epoch 34 batch 10/20, time=0.189
1416 blocks filled
3444 blocks filled
823 blocks filled
1695 blocks filled
772 blocks filled
1900 blocks filled
1375 blocks filled
1253 blocks filled
iou:  0.6162099
###################
adaptation
Train: epoch 35 batch 0/80, loss = 0.179, IoU1 = 0.649, IoU2 = 0.619, time = 0.859
Train: epoch 35 batch 10/80

Train: epoch 42 batch 60/80, loss = 0.186, IoU1 = 0.618, IoU2 = 0.570, time = 0.799
Train: epoch 42 batch 70/80, loss = 0.211, IoU1 = 0.648, IoU2 = 0.618, time = 0.769
###################
testing
Test: epoch 42 batch 0/20, time=0.196
Test: epoch 42 batch 10/20, time=0.188
1491 blocks filled
3406 blocks filled
961 blocks filled
1660 blocks filled
735 blocks filled
1862 blocks filled
1380 blocks filled
1512 blocks filled
iou:  0.61718243
###################
adaptation
Train: epoch 43 batch 0/80, loss = 0.163, IoU1 = 0.673, IoU2 = 0.641, time = 0.855
Train: epoch 43 batch 10/80, loss = 0.174, IoU1 = 0.620, IoU2 = 0.612, time = 0.778
Train: epoch 43 batch 20/80, loss = 0.192, IoU1 = 0.631, IoU2 = 0.596, time = 0.774
Train: epoch 43 batch 30/80, loss = 0.153, IoU1 = 0.669, IoU2 = 0.628, time = 0.770
Train: epoch 43 batch 40/80, loss = 0.145, IoU1 = 0.658, IoU2 = 0.613, time = 0.768
Train: epoch 43 batch 50/80, loss = 0.173, IoU1 = 0.667, IoU2 = 0.626, time = 0.768
Train: epoch 43 batch 60/8

iou:  0.60413843
###################
adaptation
Train: epoch 51 batch 0/80, loss = 0.178, IoU1 = 0.673, IoU2 = 0.603, time = 0.854
Train: epoch 51 batch 10/80, loss = 0.148, IoU1 = 0.646, IoU2 = 0.599, time = 0.771
Train: epoch 51 batch 20/80, loss = 0.165, IoU1 = 0.629, IoU2 = 0.568, time = 0.770
Train: epoch 51 batch 30/80, loss = 0.294, IoU1 = 0.628, IoU2 = 0.583, time = 0.770
Train: epoch 51 batch 40/80, loss = 0.138, IoU1 = 0.665, IoU2 = 0.618, time = 0.768
Train: epoch 51 batch 50/80, loss = 0.142, IoU1 = 0.681, IoU2 = 0.638, time = 0.770
Train: epoch 51 batch 60/80, loss = 0.128, IoU1 = 0.705, IoU2 = 0.657, time = 0.772
Train: epoch 51 batch 70/80, loss = 0.145, IoU1 = 0.699, IoU2 = 0.652, time = 0.769
###################
testing
Test: epoch 51 batch 0/20, time=0.192
Test: epoch 51 batch 10/20, time=0.187
1488 blocks filled
3880 blocks filled
909 blocks filled
1914 blocks filled
736 blocks filled
1949 blocks filled
1482 blocks filled
1463 blocks filled
iou:  0.60199934
#########

In [ ]:
print(best_iou,best_epoch)

generator.save_parameters("model/generator of GA on shapenet "+opt.cls)